<h2 style="font-weight: bold">Titanic Competition</h2>

<h4>This is my first published notebook ever, So yeah it can't be about something other than the Titanic Competition 😄<br><br>I will be doing a simple then advanced EDA, Data Visualization and Pre-Processing. I also will test different models and techniques to improve my score.<br></h4>

* <h5 style="font-weight: 700">Your feedback is very welcome</h5>
* <h5 style="font-weight: 700">If you find this notebook useful, please don't forget to upvote it!</h5>


In [1]:
# Required packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

In [2]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')

In [3]:
train.head()

#  **Exploratory Data Analysis**

In [4]:
# Getting to know data
print(train.shape)
print(test.shape)

In [5]:
# summary of numerical variable
train.describe()

In [6]:
train.describe()

In [7]:
# summary of categorial variable
train.info()

In [8]:
test.info()

In [9]:
# checking null values
print("training data\n",train.isnull().sum())
print("\ntesting data\n",test.isnull().sum())

In [10]:
# let's clean visualizations :)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [11]:
# Visualization
count_plt = sns.countplot(train["Survived"])
plt.show(count_plt)

Sex_plt = sns.countplot(x= "Survived",data=train, hue="Sex")
plt.show(Sex_plt)

Embarked_plt = sns.countplot(x="Survived", data=train, hue="Embarked")
plt.show(Embarked_plt)

Pclass_plt = sns.countplot(x="Survived", data=train, hue="Pclass")
plt.show(Pclass_plt)

SibSp_plt = sns.boxplot(x="SibSp", y= "Survived", data=train)
plt.show(SibSp_plt)

Parch_plt = sns.boxplot(x="Parch", y= "Survived", data=train)
plt.show(Parch_plt)

Age_plt = sns.distplot(train["Age"])
plt.show(Age_plt)

In [12]:
# Correlation heatmap
f, ax = plt.subplots(figsize=(10, 8))
corr = train.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool_),linewidths=0.1,annot=True, cmap=sns.diverging_palette(150, 10, as_cmap=True),
            square=True, ax=ax)

# **Pre-Processing**

In [13]:
# age_mean, fair mean
train['Age'].fillna(train['Age'].mean(), inplace = True)
test['Age'].fillna(train['Age'].mean(), inplace = True)
train['Embarked'].fillna('S', inplace = True)
test['Fare'].fillna(train['Fare'].mean(), inplace = True)

In [14]:
train.isnull().sum()

In [15]:
test.isnull().sum()

In [16]:
# drop cabin column
train.drop(columns=["Cabin", "Name"], axis=1, inplace=True)
test.drop(columns=["Cabin", "Name"], axis=1, inplace=True)

In [17]:
# Removing insignificant variables

# change categorical to int for train
embarked=pd.get_dummies(train["Embarked"], drop_first=True)
sex=pd.get_dummies(train["Sex"], drop_first=True)

train=pd.concat([train, embarked, sex],axis=1)
train.drop(columns=["Sex", "Embarked", "Ticket"], axis=1, inplace=True)

# change categorical to int for test
embarked=pd.get_dummies(test["Embarked"], drop_first=True)
sex=pd.get_dummies(test["Sex"], drop_first=True)

test=pd.concat([test, embarked, sex],axis=1)
test.drop(columns=["Sex", "Embarked", "Ticket"], axis=1, inplace=True)

In [18]:
# checking data
print(train.head())
print(test.head())
print(train.corr())

In [19]:
train.info()

In [20]:
test.info()

# **Training and Predicting**

In [21]:
# Classification Methods

# Logistic Regression
train_x= train.drop(columns=["Survived"], axis=1)
train_y= train["Survived"]

test_x= test.copy()

logistic = LogisticRegression(solver='liblinear')
logistic.fit(train_x, train_y)

prediction_y= logistic.predict(test_x)

print(logistic.score(train_x, train_y))

In [22]:
# Sorting k nearest neighbours classifier

KNN_classifier = KNeighborsClassifier(n_neighbors=3)
KNN_classifier.fit(train_x, train_y)
prediction_y = KNN_classifier.predict(test_x)

print(KNN_classifier.score(train_x, train_y))

In [23]:
# Random forest

model_RF = RandomForestClassifier(n_estimators=100, max_depth=4, min_samples_split=2, min_samples_leaf=2, random_state=2)
model_RF.fit(train_x, train_y)
prediction_y = model_RF.predict(test_x)

print(model_RF.score(train_x, train_y))

In [24]:
# Gradient Boosting

model_GB = GradientBoostingClassifier(min_samples_leaf=5,max_leaf_nodes=5)
model_GB.fit(train_x, train_y)
prediction_y = model_GB.predict(test_x)

print(model_GB.score(train_x, train_y))

In [25]:
# Submission

output= pd.DataFrame({"PassengerId": test["PassengerId"], "Survived": prediction_y})
output.to_csv("Submission.csv", index=False)
print("Done")